In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import datetime

In [2]:
data_file_name = 'face_data_w.npz'
test_file_name = 'test_data_w.npz'

learning_rate = 0.000001
epochs = 2000
batch_size = 50

In [3]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
            
            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 80, 55, 1])
            self.Y = tf.placeholder(tf.float32, [None, 2])
            
            # Convolution Layer 1 and Pooling Layer 1
            conv1 = tf.layers.conv2d(
                inputs = self.X,
                filters = 32,
                kernel_size = [5, 5],
                padding = 'SAME',
                activation = tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(
                inputs = conv1,
                pool_size = [2, 2],
                strides = 2,
                padding = 'SAME')
            dropout1 = tf.layers.dropout(
                inputs = pool1,
                rate = 0.7,
                training = self.training)
            
            # Convolution Layer 2 and Pooling Layer 2
            conv2 = tf.layers.conv2d(
                inputs = dropout1, 
                filters = 128,
                kernel_size = [3, 3],
                padding = 'SAME',
                activation = tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(
                inputs = conv2,
                pool_size = [2, 2],
                strides = 2,
                padding = 'SAME')
            dropout2 = tf.layers.dropout(
                inputs = pool2,
                rate = 0.7,
                training = self.training)

#             # Convolution Layer 3 and Pooling Layer 3
#             conv3 = tf.layers.conv2d(
#                 inputs = dropout2, 
#                 filters = 128,
#                 kernel_size = [3, 3],
#                 padding = 'SAME',
#                 activation = tf.nn.relu)            
#             pool3 = tf.layers.max_pooling2d(
#                 inputs = conv3,
#                 pool_size = [2, 2],
#                 strides = 2,
#                 padding = 'SAME')     
#             dropout3 = tf.layers.dropout(
#                 inputs = pool3,
#                 rate = 0.7,
#                 training = self.training)

            # Dense Layer
            flat = tf.reshape(dropout2, [-1, 20 * 14 * 128])
            dense4 = tf.layers.dense(
                inputs = flat,
                units = 625,
                activation = tf.nn.relu)
            dropout4 = tf.layers.dropout(
                inputs = dense4,
                rate = 0.5,
                training = self.training)

            # Logits Layer (FC 512 inputs -> 2 outputs)
            self.logits = tf.layers.dense(
                inputs = dropout4,
                units = 2)

        # Define cost, optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
    def predict(self, x_test, training = False):
        return self.sess.run(
            self.logits,
            feed_dict = {self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training = False):
        return self.sess.run(
            self.acc,
            feed_dict = {self.X: x_test, self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training = True):
        return self.sess.run(
            [self.cost, self.optimizer],
            feed_dict = {self.X: x_data, self.Y: y_data, self.training: training})

In [4]:
if os.path.isfile(data_file_name):
    file = np.load(data_file_name)
    data = file['data']
    labels = file['sex_labels']
else:
    print("No data file.")
    sys.exit()
    
if os.path.isfile(test_file_name):
    file = np.load(test_file_name)
    td = file['data']
    tl = file['sex_labels']
else:
    print("No test file")
    sys.exit()

# data = np.concatenate((data, td), axis = 0)
# labels = np.concatenate((labels, tl), axis = 0)
print("data load success")

data load success


In [5]:
sess = tf.Session()

models = []
num_models = 5
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())


# train models
print('Learning started at ', datetime.datetime.now().time())

for m_idx, m in enumerate(models):
    avg_cost_list = np.zeros(len(models))
    acc_list = np.zeros(len(models))
    
    idx = np.random.permutation(len(data))

    idx_tr = idx[0:int(0.8 * len(data))]
    idx_va = idx[int(0.8 * len(data)):]

    x_tr = data[idx_tr, :]
    y_tr = labels[idx_tr, :]
    

    x_va = data[idx_va, :]
    y_va = labels[idx_va, :]
    
    print('Model ', '%1d' % (m_idx + 1))
    
    for epoch in range(epochs):
        avg_cost_list[m_idx] = 0
        total_batch = int(len(x_tr) / batch_size)
        training_batch = zip(range(0, len(x_tr), batch_size), range(batch_size, len(x_tr) + 1, batch_size))
        for start, end in training_batch:
            c, _ = m.train(x_tr[start:end], y_tr[start:end])
            avg_cost_list[m_idx] += c / total_batch
        acc_list[m_idx] = m.get_accuracy(x_va, y_va)    
        print('Model ', '%1d' % (m_idx + 1), 'epoch: ', '%04d' % (epoch + 1))
        print('cost = ', avg_cost_list[m_idx], '\naccuracy = ', acc_list[m_idx])
        print("   ")

print('Learning Finished at ', datetime.datetime.now().time())

Learning started at  21:50:22.205695
Model  1
Model  1 epoch:  0001
cost =  64.6195306778 
accuracy =  0.419354826212
   
Model  1 epoch:  0002
cost =  60.0585287412 
accuracy =  0.519354820251
   
Model  1 epoch:  0003
cost =  58.0444003741 
accuracy =  0.487096786499
   
Model  1 epoch:  0004
cost =  62.5618832906 
accuracy =  0.45806452632
   
Model  1 epoch:  0005
cost =  56.8625880877 
accuracy =  0.56129026413
   
Model  1 epoch:  0006
cost =  61.5231568019 
accuracy =  0.490322589874
   
Model  1 epoch:  0007
cost =  51.7877659798 
accuracy =  0.49354839325
   
Model  1 epoch:  0008
cost =  49.8682107925 
accuracy =  0.49354839325
   
Model  1 epoch:  0009
cost =  55.6415688197 
accuracy =  0.451612859964
   
Model  1 epoch:  0010
cost =  51.8033989271 
accuracy =  0.483870983124
   
Model  1 epoch:  0011
cost =  53.0214454333 
accuracy =  0.558064520359
   
Model  1 epoch:  0012
cost =  52.5523719788 
accuracy =  0.474193513393
   
Model  1 epoch:  0013
cost =  53.7236946424 
a

In [6]:
# test models and check accuracy
test_size = len(td)
predictions = np.zeros(test_size * 2).reshape(test_size, 2)

for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy: ', m.get_accuracy(td, tl))
    p = m.predict(td)
    predictions += p
    
ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(tl, 1))
ensemble_accuracy = (tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32)))
print('Ensemble accuracy: ', sess.run(ensemble_accuracy))

0 Accuracy:  0.590206
1 Accuracy:  0.53866
2 Accuracy:  0.592783
3 Accuracy:  0.590206
4 Accuracy:  0.618557
Ensemble accuracy:  0.587629
